In [8]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
from FeatureExtraction import extract_features
import warnings
warnings.filterwarnings("ignore")
import time

#path to training data
source   = './data/'   

#path where models for each speaker will be saved
modelpath = "models/"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian gender Models
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname in gmm_files]


In [10]:
error = 0
total_sample = 0.0
print()
take = int(input("Use Input Audio: Press '1' or Run Test: Press '0' ?"))
if take == 1:
    print()
    path = input("Enter the File Name:")
    print("Testing Audio : ", path)    
    sr,audio = read(source + path)
    vector   = extract_features(audio,sr)

    log_likelihood = np.zeros(len(models)) 

    for i in range(len(models)):
        gmm = models[i] 
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    
    winner = np.argmax(log_likelihood)
    print("\tdetected as - ", speakers[winner])

    time.sleep(1.0)
elif take == 0:
    test_file = "testSamplePath.txt"        
    file_paths = open(test_file,'r')
    for path in file_paths:   
        total_sample += 1.0
        path = path.strip()   
        print("File: ", path)
        sr,audio = read(source + path)
        vector = extract_features(audio,sr)
        log_likelihood = np.zeros(len(models)) 
        for i in range(len(models)):
            gmm = models[i]
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()
    
        winner = np.argmax(log_likelihood)
        print( "\tSpeaker - ", speakers[winner], "\n")
        if ("atlas" in path and speakers[winner] != "Jack") or "atlas" not in path and speakers[winner] == "Jack":
            error += 1
            time.sleep(1.0)
    print("Errors: {0} | Total Samples: {1}".format(error, total_sample))
    accuracy = ((total_sample - error) / total_sample) * 100
    print("Accuracy with MFCC + GMM: ", accuracy, "%")


Use Input Audio: Press '1' or Run Test: Press '0' ?0
File:  atlas-17.wav
	Speaker -  Jack 

File:  atlas-23.wav
	Speaker -  Jack 

File:  atlas-27.wav
	Speaker -  Jack 

File:  atlas-29.wav
	Speaker -  Jack 

File:  noah-4.wav
	Speaker -  Other 

File:  noah-6.wav
	Speaker -  Other 

File:  subrat-2.wav
	Speaker -  Other 

File:  subrat-4.wav
	Speaker -  Other 

File:  vamsi-3.wav
	Speaker -  Other 

File:  vamsi-5.wav
	Speaker -  Other 

Errors: 0 | Total Samples: 10.0
Accuracy with MFCC + GMM:  100.0 %
